In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyProtein
from msmrd2.integrators import overdampedLangevin as odLangevin

In [ ]:
# Define two particle list of rod-like particles
numparticles = 2
D = 1.0
Drot = 1.0
pyPartlist = [] 
positionList = [np.array([1.0,0,0]), np.array([-1.,0.,0.])]
orientationList = [ np.array([1.,0.,0.,0.]), np.array([np.cos(np.pi/2),0.,0.,np.sin(np.pi/2)])]
for i in range(numparticles):           
    part = msmrd2.particle(D, Drot, positionList[i], orientationList[i])
    part.setType(i)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# Note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = 0 # Seed = -1 used random device as seed
bodytype = 'rigidbodyMix' # Mix indicates there can be different types of rigidbody particles
integrator = odLangevin(dt, seed, bodytype) 
integrator.setKbT(1.0)

In [ ]:
# Define boundary (choose either spherical or box)
radius = 5.0
sphereBoundary = msmrd2.sphere(radius,'reflective')
integrator.setBoundary(sphereBoundary)

In [ ]:
# Define Patchy Protein potential
sigma = 1.0
strength = 65
patchesCoordinates1 = [np.array([1.,0.,0.]), 
                       np.array([0.,1.,0.]),
                       np.array([-1.,0.,0.]),
                       np.array([0.,-1.,0.]),
                       np.array([0.,0.,1.]),
                       np.array([0.,0.,-1.])]
patchesCoordinates2 = [np.array([1.,0.,0.])]
potentialPatchyProtein = patchyProtein(sigma, strength, patchesCoordinates1, patchesCoordinates2)
integrator.setPairRigidBodyMixPotential(potentialPatchyProtein)

In [ ]:
# Integrate particle list and print only positions 
timesteps = 50000000
stride = 1000 #1000
datafile  = open('../data/vmd/patchyProteins.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(9) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            if part.type == 0:
                v0 = part.position
                v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates1[0], part.orientation)
                v2 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[1], part.orientation)
                v3 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[2], part.orientation)
                v4 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[3], part.orientation)
                v5 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[4], part.orientation)
                v6 = v0 + 0.3*sigma*quats.rotateVec(patchesCoordinates1[5], part.orientation)
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v3)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v4)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v5)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v6)) + '\n')
            if part.type == 1:
                v0 = part.position
                v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyProteins(numparticles = numparticles)
print("Percentage complete: ", 100, " %")

In [ ]:
# Calculate potential for aligned and misaligned molecules
dist = np.arange(0, 2*sigma, 0.01)
pos1 = np.array([0., 0., 0.])
pos2 = np.array([[i, 0., 0.] for i in dist])
phi1 = 0.0
phi2 = np.pi
phi1p = np.pi
phi2p = 0.0
u = np.array([0.,0.,1.]) #rotationAxis
# Quaternions that rotate phi along axis u
theta1 = np.array([np.cos(0.5*phi1 ),
                   np.sin(0.5*phi1 )*u[0],
                   np.sin(0.5*phi1 )*u[1],
                   np.sin(0.5*phi1 )*u[2]]) 
theta2 = np.array([np.cos(0.5*phi2 ),
                   np.sin(0.5*phi2 )*u[0],
                   np.sin(0.5*phi2 )*u[1],
                   np.sin(0.5*phi2 )*u[2]]) 
theta1p = np.array([np.cos(0.5*phi1p ),
                   np.sin(0.5*phi1p )*u[0],
                   np.sin(0.5*phi1p )*u[1],
                   np.sin(0.5*phi1p )*u[2]]) 
theta2p = np.array([np.cos(0.5*phi2p ),
                   np.sin(0.5*phi2p )*u[0],
                   np.sin(0.5*phi2p )*u[1],
                   np.sin(0.5*phi2p )*u[2]]) 

potantialAligned = [potentialPatchyProtein.evaluate(pos1,p2,theta1,theta2, 0, 1) for p2 in pos2]
potantialMisaligned = [potentialPatchyProtein.evaluate(pos1,p2,theta1p,theta2p, 0, 1) for p2 in pos2]

In [ ]:
# Plot potential
plt.plot(dist, potantialAligned, label = "aligned")
plt.plot(dist, potantialMisaligned, label = "misaligned")
plt.plot(dist, 0*dist, '--k', lw=0.5)
plt.legend()
plt.xlim([0,2*sigma])

In [ ]:
potentialPatchyProtein.forceTorque(pos1,pos2[20],theta1p,theta2p, 0, 1)